# Machine Learning Zoomcamp

# Homework 3

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [64]:
url = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'
df = pd.read_csv(url)

df.head(5)

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [65]:
# Show the information of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


## Data preperation 

In [66]:
# check for missing values
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [67]:
# Replace categorical feature with 'NA'
categorical_features = [
    'lead_source',
    'industry',
    'employment_status',
    'location'
]

for feature in categorical_features:
    df[feature] = df[feature].fillna('NA')


In [68]:
# Replace numerical feature with 0.0
numerical_features = [
    'annual_income'
]

for feature in numerical_features:
    df[feature] = df[feature].fillna(0.0)

## Question 1

What is the most frequent observation (mode) for the column industry?

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1462 non-null   object 
 1   industry                  1462 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1462 non-null   float64
 4   employment_status         1462 non-null   object 
 5   location                  1462 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [70]:
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

In [71]:
# Show the most frequent observation (mode) for the column industry
df['industry'].mode()

0    retail
Name: industry, dtype: object

In [72]:
# Answer is retail

##  Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

    * interaction_count and lead_score
    * number_of_courses_viewed and lead_score
    * number_of_courses_viewed and interaction_count
    * annual_income and interaction_count




In [73]:
numerical_features = [
    'number_of_courses_viewed',
    'annual_income',
    'interaction_count',
    'lead_score',
    'converted'
]

### 2.1 correlation for interaction_count and lead_score

In [74]:
corr_interaction_count_lead_score = df['interaction_count'].corr(df['lead_score'])

print(f"Correlation between interaction_count and lead_score: {corr_interaction_count_lead_score}")

Correlation between interaction_count and lead_score: 0.009888182496913077


### 2.2 Correlation for number_of_courses_viewed and lead_score

In [75]:
corr_number_of_courses_viewed_lead_score = df['number_of_courses_viewed'].corr(df['lead_score'])
print(f"Correlation between number_of_courses_viewed and lead_score: {corr_number_of_courses_viewed_lead_score}")

Correlation between number_of_courses_viewed and lead_score: -0.00487899835468127


### 2.3 Correlation for number_of_courses_viewed and interaction_count

In [76]:
corr_number_of_courses_viewed_interaction_count = df['number_of_courses_viewed'].corr(df['interaction_count'])
print(f"Correlation between number_of_courses_viewed and interaction_count: {corr_number_of_courses_viewed_interaction_count}")

Correlation between number_of_courses_viewed and interaction_count: -0.023565222882888103


### 2.4 Coorelation for annual_income and interaction_count

In [77]:
corr_annual_income_interaction_count = df['annual_income'].corr(df['interaction_count'])
print(f"Correlation between annual_income and interaction_count: {corr_annual_income_interaction_count}")

Correlation between annual_income and interaction_count: 0.027036472404814337


In [78]:
# Get the biggest correlation
print(f"Biggest correlation: {max(corr_interaction_count_lead_score, corr_number_of_courses_viewed_lead_score, corr_number_of_courses_viewed_interaction_count, corr_annual_income_interaction_count)}")

Biggest correlation: 0.027036472404814337


In [79]:
# Answer is annual_income and interaction_count

## Question 3

Which of these variables has the biggest mutual information score?

In [80]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

X = df.drop(columns=['converted'])
y = df['converted']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_features = ['lead_source', 'industry', 'employment_status', 'location']

mi_scores = {}

for col in categorical_features:
    le = LabelEncoder()
    encoded = le.fit_transform(X_train[col])
    mi = mutual_info_classif(encoded.reshape(-1, 1), y_train, discrete_features=True)
    mi_scores[col] = round(mi[0], 2)

mi_df = pd.DataFrame(mi_scores.items(), columns=['Feature', 'Mutual Information']).sort_values(by='Mutual Information', ascending=False)
print(mi_df)


             Feature  Mutual Information
0        lead_source                0.03
1           industry                0.01
2  employment_status                0.01
3           location                0.00


## Queastion 4

What accuracy did you get?

In [81]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

X = df.drop(columns=['converted'])
y = df['converted']

numeric_features = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
categorical_features = ['lead_source', 'industry', 'employment_status', 'location']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)

accuracy = round(accuracy_score(y_val, y_pred), 2)
print("Validation Accuracy:", accuracy)


Validation Accuracy: 0.86


## Question 5 

Which of following feature has the smallest difference?

In [82]:
baseline_acc = accuracy_score(y_val, clf.predict(X_val))
print("Baseline accuracy:", baseline_acc)

Baseline accuracy: 0.8561643835616438


In [83]:
feature_sets = numeric_features + categorical_features
diffs = {}


for feature in feature_sets:
    X_reduced = X.drop(columns=[feature])
    preprocessor_reduced = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, [f for f in numeric_features if f != feature]),
            ('cat', categorical_transformer, [f for f in categorical_features if f != feature])
        ],
        remainder='drop'
    )
    clf_reduced = Pipeline(steps=[('preprocessor', preprocessor_reduced), ('model', model)])
    clf_reduced.fit(X_train[X_reduced.columns], y_train)
    acc = accuracy_score(y_val, clf_reduced.predict(X_val[X_reduced.columns]))
    diffs[feature] = baseline_acc - acc

for f, d in diffs.items():
    print(f"{f}: {d:.4f}")


number_of_courses_viewed: 0.1164
annual_income: 0.0068
interaction_count: 0.1473
lead_score: 0.0068
lead_source: 0.0582
industry: 0.0000
employment_status: 0.0137
location: 0.0103


## Question 6

Which of these C leads to the best accuracy on the validation set?

In [84]:
C_values = [0.01, 0.1, 1, 10, 100]
results = {}

for c in C_values:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    clf = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    acc = round(accuracy_score(y_val, y_pred), 3)
    results[c] = acc

for c, acc in results.items():
    print(f"C={c}: Validation Accuracy={acc}")

C=0.01: Validation Accuracy=0.849
C=0.1: Validation Accuracy=0.856
C=1: Validation Accuracy=0.856
C=10: Validation Accuracy=0.856
C=100: Validation Accuracy=0.856
